# A preliminary approach to corpus driven research of Persian classical music
Babak Nikzat and Rafael Caro Repetto

Simple tools for the analysis of the **KUG Dastgāh Corpus** (**KDC**).

In [ ]:
import essentia
import essentia.standard as es
import essentiaUtils as eu
import matplotlib.pyplot as plt
import numpy as np
import IPython
import intonation
import os

In the following cell, specify in the `kdc_folder` variable the path to the `KDC` folder where the **KDC** corpus is locally stored in your computer. By default, the code assumes it is in the parent folder to the one where this notebook is stored.

In [ ]:
# Specify the path to the KDC folder in the following variable
kdc_folder = '../KDC'

data_path = os.path.join(kdc_folder, 'KDC-data.csv')

with open(data_path, 'r') as f:
    kdc_data = f.readlines()

In the following cell you can specify which recordings from the **KDC** you would like to analyse. Just include the list of items you would like to study in the `search_items` variable. By default, the code search for the given items in the `Dastgah` column from the `KDC-data.csv` file. If required, a different search criteria can be specified in the `search_column` variable (for example, `Gushe` or `Instrument`).

In [ ]:
# Specify the items to be searched as a list in the following variable
search_items = ['Shur']
# Specify the category of items to be searched
search_column = 'Dastgah'

search_column_index = None
heading = kdc_data[0].rstrip().split(';')
for i in range(len(heading)):
    if search_column == heading[i]:
        search_column_index = i
if search_column_index == None:
    print('ERROR: no search column has been found')

recordings = {}
    
for row in kdc_data[1:]:
    row_data = row.rstrip().split(';')
    if row_data[search_column_index] in search_items:
        recording_path = os.path.join(kdc_folder, 'KDC-'+row_data[1], row_data[0])
        shahed = row_data[7]
        minf0 = row_data[8]
        maxf0 = row_data[9]
        f0_cf = row_data[10]
        if shahed == '':
            shahed = None
        if minf0 == '':
            minf0 = '20'
        if maxf0 == '':
            maxf0 = '22050'
        if f0_cf == '':
            f0_cf = '0.9'
        recordings[recording_path] = {'shahed': shahed, 'minf0':float(minf0), 'maxf0':float(maxf0), 'f0_cf':float(f0_cf)}

print('{} recordings found:'.format(len(recordings)))
recordingsPaths = list(recordings.keys())
for i in range(len(recordingsPaths)):
    print('    {}: {}'.format(i, recordingsPaths[i].split('/')[-1]))

## Analysis of single track

### Compute pitch track
The pitch track is a necessary step for the computation of pitch histograms, the vibrato analysis and plotting the loudness related figures.

#### Load audio
To select on the recordings retrieved in the previous cell, specify the corresponding index in the following to the variable `recording_index`.

In [ ]:
# Specify the index to the recording to be analysed in the following variable
recording_index = 0

recordingPath = recordingsPaths[recording_index]
recording = recordings[recordingPath]

print(recordingPath.split('/')[-1])

IPython.display.Audio(recordingPath)

In [ ]:
loader = es.MonoLoader(filename=recordingPath)
eqLoud = es.EqualLoudness()

audio = eqLoud(loader())

#### Extract pitch track

In [ ]:
windowSize = 2048
hopSize = 128

In [ ]:
f0 = eu.get_f0(audio, minf0=recording['minf0'], maxf0=recording['maxf0'], cf=recording['f0_cf'], ws=windowSize, hs=hopSize)
timeStamps = np.arange(f0.size)*hopSize/44100

#### Plot with spectrogram

In [ ]:
x, y, z = eu.spectrogram(audio, ws=windowSize, hs=hopSize)

Plotting the spectrogram might take few minutes. To zoom in to a particular region of the plot, specify a limit to the x and y axes. The start and end values of the limit in each axis should be given as a tuple (for example, `lim_y = (20, 1500)`).

In [ ]:
# Limits to the x and y axis can be specified in the following variables
lim_x = None    # Needs a tuple
lim_y = None    # Needs a tuple

plt.rcParams['figure.figsize'] = (15, 6)

plt.pcolormesh(x, y, z)
plt.plot(timeStamps[f0>0], f0[f0>0], '.k', markersize=0.5)
plt.xlim(lim_x)
plt.ylim(lim_y)
plt.show()

### Compute histogram (Hz)

In [ ]:
pitch_obj = intonation.Pitch(timeStamps[f0>20], f0[f0>0])

In [ ]:
rec_obj = intonation.Recording(pitch_obj)

In [ ]:
rec_obj.compute_hist(bins=1000)
rec_obj.histogram.get_peaks()
rec_obj.histogram.plot()

In [ ]:
print(recordingPath.split('/')[-1])
print()

peaks = rec_obj.histogram.peaks['peaks']
for i in range(len(peaks[0])):
    print('{}. {} : {:.6f}'.format(i, round(peaks[0][i]), peaks[1][i]))

### Compute histogram (cents)

In [ ]:
shahed = recording['shahed']
one_octave = False

if shahed == None:
    print('ERROR: There is no shahed computed for this recording')
else:
    print('Shahed for {}: {}'.format(recordingPath.split('/')[-1], shahed))

In [ ]:
f0cents = 1200 * np.log2(f0[f0>0] / float(shahed))

In [ ]:
pitch_cents_obj = intonation.Pitch(timeStamps[f0>0], f0cents)

In [ ]:
rec_cents_obj = intonation.Recording(pitch_cents_obj)

In [ ]:
rec_cents_obj.compute_hist(bins=1200, folded=one_octave)
rec_cents_obj.histogram.get_peaks()
rec_cents_obj.histogram.plot()

In [ ]:
print(recordingPath.split('/')[-1])
print()

if one_octave:
    print('   0 : {:.6f}'.format(rec_cents_obj.histogram.y[0]))

peaks_cents = rec_cents_obj.histogram.peaks['peaks']
for i in range(len(peaks_cents[0])):
    print('{}. {} : {:.6f}'.format(i, round(peaks_cents[0][i]), peaks_cents[1][i]))

### Vibrato analysis

In [ ]:
vib = es.Vibrato(sampleRate = 44100/hopSize)
vibF, vibE = vib(essentia.array(f0))

To zoom in to a particular region of the plot, specify a limit to the x and y axes. The start and end values of the limit in each axis should be given as a tuple (for example, `lim_x = (0, 10)`).

In [ ]:
# Limits to the x and y axes can be specified in the following variables
lim_x = None    # Needs a tuple
lim_y = None    # Needs a tuple

plt.rcParams['figure.figsize'] = (15, 6)

plt.figure()
plt.subplot(311)
plt.plot(timeStamps[f0>0], f0[f0>0], '.k', markersize=0.5)
plt.xlim(lim_x)
plt.ylim(lim_y)
plt.title('f0')
plt.subplot(312)
plt.plot(timeStamps, vibF)
plt.xlim(lim_x)
plt.title('frequency')
plt.subplot(313)
plt.plot(timeStamps, vibE)
plt.xlim(lim_x)
plt.title('extent')
plt.tight_layout()
plt.show()

In [ ]:
print(recordingPath.split('/')[-1])
print()

freq_mean = np.mean(vibF[vibF>0])
freq_sd = np.std(vibF[vibF>0])
print('Frequency:\tmean: {:.2f} Hz\t\tSD: {:.2f} Hz'.format(freq_mean, freq_sd))

ex_mean = np.mean(vibE[vibE>20])
ex_sd = np.std(vibE[vibE>20])
print('Extent:\t\tmean: {:.2f} cents\tSD: {:.2f} cents'.format(ex_mean, ex_sd))

### Loudness analysis

In [ ]:
loud = es.Loudness()
energy = es.Energy()
rms = es.RMS()

loudTrack = []
energyTrack = []
rmsTrack = []

for frame in es.FrameGenerator(audio, frameSize=2048, hopSize=hopSize):
    frameLoud = loud(frame)
    frameEnergy = energy(frame)
    frameRms = rms(frame)
    loudTrack.append(frameLoud)
    energyTrack.append(frameEnergy)
    rmsTrack.append(frameRms)

In [ ]:
lim_x = None       # Needs a tuple
lim_y = None         # Needs a tuple
lim_y_loud = None    # Needs a tuple

plt.rcParams['figure.figsize'] = (15, 6)

plt.figure()
plt.subplot(411)
plt.plot(timeStamps, f0, '.k', markersize=0.5)
plt.xlim(lim_x)
plt.ylim(lim_y)
plt.title('pitch')
plt.subplot(412)
plt.plot(timeStamps, loudTrack)
plt.xlim(lim_x)
plt.ylim(lim_y_loud)
plt.title('loudness (Loudness)')
plt.subplot(413)
plt.plot(timeStamps, energyTrack)
plt.xlim(lim_x)
plt.ylim(lim_y_loud)
plt.title('loudness (Energy)')
plt.subplot(414)
plt.plot(timeStamps, rmsTrack)
plt.xlim(lim_x)
plt.ylim(lim_y_loud)
plt.title('loudness (RMS)')
plt.tight_layout()
plt.show()

### Start and end analysis
This block of code retrives the pitch value of the first note of the given pitch track, the pitch value of the first note that lasts longer than 1 second, and the pitch value of the last note, and compares them with the pitch value of the recording's *shahed*.

In [ ]:
pitchContourSegmentation = es.PitchContourSegmentation()

midiNotes = pitchContourSegmentation(essentia.array(f0), audio)

To dump the retrieve notes to a text file, so that it can be used in other software, like Sonic Visualiser, change the `print_notes` variable to `True`.

In [ ]:
# Change the following variable to True in order to dump the notes returned in the previous cell to a text file
print_notes = False

if print_notes:
    txt = ''
    for i in range(len(midiNotes[0])):
        txt += str(midiNotes[0][i]) + '\t' + str(midiNotes[1][i]) + '\t' + midi2Hz(midiNotes[2][i]) + '\n'

    with open(recordingPath[:-5]+'-notes.txt', 'w') as f:
        f.write(txt.rstrip())

In [ ]:
def midi2Hz(midi):
    '''
    Converts a given midi value to its equivalent in hertz
    
    Args:
        midi (str): midi value as string
        
    Returns:
        frequency (str): equivalent frequency in Hertz as string
        
    >>> midi2Hz('60')
    '261.6255653005986'
    '''
    f = 2 ** ((float(midi) - 69) / 12.) * 440
    return str(f)

In [ ]:
for i in range(10):
    print('{:.2f}: {:.2f}'.format(midiNotes[1][i], midiNotes[2][i]))

In [ ]:
first_start = midiNotes[0][0]
first_end = first_start + midiNotes[1][0]

i = 0
j = 0
dur = midiNotes[1][0]
while dur < 1:
    i += 1
    j = 0
    dur = midiNotes[1][i]
    midi = midiNotes[2][i]
    while midiNotes[2][i+j+1] == midi:
        dur += midiNotes[1][i+j+1]
        j += 1
first_long_start = midiNotes[0][i]
first_long_end = first_long_start + dur

last_start = midiNotes[0][-1]
last_end = last_start + midiNotes[1][-1]

first_start_i = np.abs(timeStamps - first_start).argmin()
first_end_i = np.abs(timeStamps - first_end).argmin()

first_long_start_i = np.abs(timeStamps - first_long_start).argmin()
first_long_end_i = np.abs(timeStamps - first_long_end).argmin()

last_start_i = np.abs(timeStamps - last_start).argmin()
last_end_i = np.abs(timeStamps - last_end).argmin()

first = f0[first_start_i:first_end_i+1]
first_long = f0[first_long_start_i:first_long_end_i+1]
last = f0[last_start_i:last_end_i+1]

In [ ]:
plt.rcParams['figure.figsize'] = (15, 6)

plt.figure()
plt.subplot(311)
plt.plot(timeStamps[first_start_i:first_end_i+1], first)
plt.h
plt.title('first')
plt.subplot(312)
plt.plot(timeStamps[first_long_start_i:first_long_end_i+1], first_long)
plt.title('first long')
plt.subplot(313)
plt.plot(timeStamps[last_start_i:last_end_i+1], last)
plt.title('last')
plt.tight_layout()
plt.show()

In [ ]:
print(recordingPath.split('/')[-1])
print('Shahed: {}'.format(recording['shahed']))
print('First note (mean): {}'.format(round(np.mean(first))))
print('First long note (mean): {}'.format(round(np.mean(first_long))))
print('Last note (mean): {}'.format(round(np.mean(last))))

## Analysis of multiple recordings

### Select recordings
Give the list of **indexes** of the recordings resulting from the third cell of this notebook. If no index is given, all those recordings will be selected by default.

In [ ]:
# Give the indexes of the recordings to be analysed in the following list. If none is given, all recordings are selected.
selected_indexes = []

selected_recordings = {}

if len(selected_indexes) == 0:
    selected_indexes = range(len(recordings.keys()))
    
for i in range(len(selected_indexes)):
    recordingPath = recordingsPaths[i]
    recording = recordings[recordingPath]
    if recording['shahed'] == None:
        print('The recording {} has no shahed: it is not selected'.format(recordingPath.split('/')[-1]))
    else:
        selected_recordings[recordingPath] = recording

print()
print('{} recordings selected:'.format(len(selected_recordings)))
for k in selected_recordings.keys():
    print('    {}'.format(k.split('/')[-1]))

### Aggregated pitch track (cents)

In [ ]:
windowSize = 2048
hopSize = 128

eqLoud = es.EqualLoudness()

agg_f0_cents = np.array([])

for k in selected_recordings.keys():
    recording = selected_recordings[k]
    loader = es.MonoLoader(filename=k)
    print('Loading {}'.format(k.split('/')[-1]))
    audio = eqLoud(loader())
    f0 = eu.get_f0(audio, minf0=recording['minf0'], maxf0=recording['maxf0'], cf=recording['f0_cf'], ws=windowSize, hs=hopSize)
    f0cents = 1200 * np.log2(f0[f0>0] / float(recording['shahed']))
    agg_f0_cents = np.append(agg_f0_cents, f0cents)
print('Done!')

### Aggregated pitch histogram (cents)

To fold the pitch histogram to a single octave, change the value of the `one_octave` variable to `True`.

In [ ]:
# Change the value of the following variable to True in order to fold the pitch histogram to one octave
one_octave=False

agg_pitch_cents_obj = intonation.Pitch(np.arange(len(agg_f0_cents[agg_f0_cents>0])), agg_f0_cents)

In [ ]:
agg_rec_cents_obj = intonation.Recording(agg_pitch_cents_obj)

In [ ]:
agg_rec_cents_obj.compute_hist(bins=1200, folded=one_octave)
agg_rec_cents_obj.histogram.get_peaks()
agg_rec_cents_obj.histogram.plot()

In [ ]:
if one_octave:
    print('   0 : {:.6f}'.format(agg_rec_cents_obj.histogram.y[0]))

agg_peaks_cents = agg_rec_cents_obj.histogram.peaks['peaks']
for i in range(len(agg_peaks_cents[0])):
    print('{}. {} : {:.6f}'.format(i, round(agg_peaks_cents[0][i]), agg_peaks_cents[1][i]))

### Start and end analysis

In [ ]:
def midi2Hz(midi):
    '''
    Converts a given midi value to its equivalent in hertz
    
    Args:
        midi (str): midi value as string
        
    Returns:
        frequency (str): equivalent frequency in Hertz as string
        
    >>> midi2Hz('60')
    '261.6255653005986'
    '''
    f = 2 ** ((float(midi) - 69) / 12.) * 440
    return str(f)

In [ ]:
windowSize = 2048
hopSize = 128

eqLoud = es.EqualLoudness()
pitchContourSegmentation = es.PitchContourSegmentation()

message = 'Recording\t\tShahed\t\tFirst note\tFirst long note\t\tLast note\n'

for  k in selected_recordings.keys():
    message += k.split('/')[-1]
    recording = selected_recordings[k]
    if len(k.split('/')[-1]) > 15:
        sep = '\t '
    else:
        sep = '\t\t '
    message += sep + selected_recordings[k]['shahed'] + '\t\t '
    loader = es.MonoLoader(filename=k)
    print('Loading {}'.format(k.split('/')[-1]))
    audio = eqLoud(loader())

    print('  Computing pitch track...')
    f0 = eu.get_f0(audio, minf0=recording['minf0'], maxf0=recording['maxf0'], cf=recording['f0_cf'], ws=windowSize, hs=hopSize)

    print('  Computing notes...')
    midiNotes = pitchContourSegmentation(essentia.array(f0), audio)

    first_start = midiNotes[0][0]
    first_end = first_start + midiNotes[1][0]
    i = 0
    j = 0
    dur = midiNotes[1][0]
    while dur < 1:
        i += 1
        j = 0
        dur = midiNotes[1][i]
        midi = midiNotes[2][i]
        while midiNotes[2][i+j+1] == midi:
            dur += midiNotes[1][i+j+1]
            j += 1
    first_long_start = midiNotes[0][i]
    first_long_end = first_long_start + dur

    last_start = midiNotes[0][-1]
    last_end = last_start + midiNotes[1][-1]

    first_start_i = np.abs(timeStamps - first_start).argmin()
    first_end_i = np.abs(timeStamps - first_end).argmin()

    first_long_start_i = np.abs(timeStamps - first_long_start).argmin()
    first_long_end_i = np.abs(timeStamps - first_long_end).argmin()

    last_start_i = np.abs(timeStamps - last_start).argmin()
    last_end_i = np.abs(timeStamps - last_end).argmin()

    first = f0[first_start_i:first_end_i+1]
    first_long = f0[first_long_start_i:first_long_end_i+1]
    last = f0[last_start_i:last_end_i+1]
    
    message += str(round(np.mean(first))) + '\t\t ' + str(round(np.mean(first_long))) + '\t\t\t ' + str(round(np.mean(last))) + '\n'

print()
print(message)

## Preprocessing steps

### Find pitch range

#### Load audio

In [ ]:
recording_index = 3

recordingPath = recordingsPaths[recording_index]
recording = recordings[recordingPath]

print(recordingPath.split('/')[-1])

IPython.display.Audio(recordingPath)

In [ ]:
loader = es.MonoLoader(filename=recordingPath)
eqLoud = es.EqualLoudness()

audio = eqLoud(loader())

#### Extract pitch track

In [ ]:
windowSize = 2048
hopSize = 128

In [ ]:
for k in recording:
    print(k + ': ' + str(recording[k]))

In [ ]:
f0 = eu.get_f0(audio, minf0=130, maxf0=300, cf=0.5, ws=windowSize, hs=hopSize)

In [ ]:
print(np.floor(min(f0[f0>0])))
print(np.ceil(max(f0)))

In [ ]:
txt = ''
for i in f0:
    txt += str(i) + '\n'

with open(recordingPath[:-5]+'-f0.txt', 'w') as f:
    f.write(txt.rstrip())

### Plot with spectrogram

In [ ]:
x, y, z = eu.spectrogram(audio, ws=windowSize, hs=hopSize)

In [ ]:
plt.rcParams['figure.figsize'] = (15, 6)

plt.pcolormesh(x, y, z)
plt.plot(x[f0>20], f0[f0>20], '.k', markersize=0.5)
plt.ylim([0, 1500])
plt.show()

## Generate pitch track for the whole dataset

In [ ]:
windowSize = 2048
hopSize = 128

In [ ]:
pt_folder = os.path.join(kdc_folder, 'pitchTracks')
try:
    os.mkdir(pt_folder)
    os.mkdir(os.path.join(pt_folder, 'KDC-CR'))
    os.mkdir(os.path.join(pt_folder, 'KDC-OR'))
except:
    print('(The pitch track folder already exists)\n')

for row in kdc_data[1:]:
    # Process data
    row_data = row.rstrip().split(';')
    col_folder = 'KDC-'+row_data[1]
    filename = row_data[0]
    print('Processing KDC-{}/{}'.format(col_folder, filename))
    recording_path = os.path.join(kdc_folder, col_folder, filename)
    minf0 = row_data[8]
    maxf0 = row_data[9]
    f0_cf = row_data[10]
    print('\tMinf0: {}\tMaxf0: {}\tf0_cf: {}'.format(minf0, maxf0, f0_cf))
    # Compute pitchtrack
    f0 = eu.get_f0(audio, minf0=130, maxf0=300, cf=0.5, ws=windowSize, hs=hopSize)
    # Save pitchtrack file
    f0_txt = ''
    f0_file = os.path.join(pt_folder, col_folder, filename[:-5]+'-f0.txt')
    for i in f0:
        f0_txt += str(i) + '\n'
    with open(f0_file, 'w') as f:
        f.write(f0_txt.rstrip())
    print('\tDone!')
print('\nAll files computed!')